In [1]:
#Here you must specify your own path
root = ''

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from tensorflow.keras.layers.experimental import preprocessing

In [3]:
# Dic to get the respective target values of the numbers.
target_dic = {
    0: '(0, 20]constant',
    1: '(20, 40]constant',
    2: '(40, 60]constant',
    3: '(60, 80]constant',
    4: '(80, 100]constant',
    5: '(0, 20]intermittent',
    6: '(20, 40]intermittent',
    7: '(40, 60]intermittent',
    8: '(60, 80]intermittent',
    9: '(80, 100]intermittent',
}

In [4]:
pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow-addons

Note: you may need to restart the kernel to use updated packages.


In [6]:
import keras_tuner as kt
from keras_tuner import BayesianOptimization
from keras_tuner import HyperModel as HM
import tensorflow_addons as tfa


In [7]:
#read in csv file into dataframe
df_train = pd.read_csv(root + 'data/02_processed/completed_train.csv', sep=';')
df_train.pop('Unnamed: 0')

#df_train.set_index('user_id',inplace=True)
df_validation = pd.read_csv(root + 'data/02_processed/completed_validation.csv', sep=';')
df_validation.set_index('user_id',inplace=True)
df_test = pd.read_csv(root + 'data/02_processed/completed_test.csv', sep=';')
df_test.set_index('user_id',inplace=True)
target_testO = df_test['target']
target_test=pd.get_dummies(df_test['target'], prefix='target')

iterator=df_train.copy()
iterator.pop('target')

#Function to transform dataframe into tf.dataset
def df_to_ds(dataframe, shuffle,batch_size):
  dataframe = dataframe.copy()
  target= pd.get_dummies(dataframe['target'], prefix='target')
    
  dataframe.pop('target')
  dataset = tf.data.Dataset.from_tensor_slices((dict(dataframe), target.values))
  if shuffle:
    dataset = dataset.shuffle(buffer_size=len(dataframe))
  dataset = dataset.batch(batch_size)
  return dataset

#Transform into tf dataset with batchsize.
batch_size = 16
train_ds = df_to_ds(df_train,shuffle=True, batch_size=batch_size)
val_ds = df_to_ds(df_validation, shuffle=False, batch_size=batch_size)
test_ds = df_to_ds(df_test, shuffle=False, batch_size=batch_size)

In [8]:
class NNHyperModel(HM):


    def build(self,hp):
     raw_inputs = {}

#Get the categorical and numerical column names
     categorical_columns=[]
     numerical_columns=[]
     for name, values in iterator.iteritems():
         if values.dtype==np.float64:
             numerical_columns.append(name)
         if values.dtype==object:
             categorical_columns.append(name)
        
     #creating the inputs for the neural network        
     for column_name in categorical_columns:
         raw_inputs[column_name] = tf.keras.Input(shape=(1,), name=column_name, dtype=tf.string)

     for column_name in numerical_columns:
         raw_inputs[column_name] = tf.keras.Input(shape=(1,), name=column_name, dtype=tf.float32)

     inputs=[]
     for name, input in raw_inputs.items():
         if input.dtype == tf.string:
          continue
        
          inputs.append(input)
            

     for name, input in raw_inputs.items():
         if input.dtype == tf.float32:
          continue
    
         lookup = preprocessing.StringLookup(vocabulary=np.unique(iterator[name]))
         x = lookup(input)
         inputs.append(x)
         print(inputs)
    
#model architecture
     HP_DROPOUT = hp.Float('dropout', 0, 0.9, step=0.1)
     HP_ACTIVATION = hp.Choice('activation',['elu', 'exponential', 'gelu', 'relu','selu'])
     HP_NEURONS=hp.Int('units', min_value = 128, max_value = 256, step = 16)
     HP_KERNELINITIALIZER = hp.Choice('kernel_initializer',['lecun_uniform','glorot_normal', 'glorot_uniform', 'he_normal','lecun_normal','' 'he_uniform','random_normal'])
     HP_LEARNINGRATE= hp.Choice('learning_rate', values = [1e-2,1e-3,1e-4,1e-5,1e-6])
     HP_l1_l2 = hp.Choice('learning_rate', values = [1e-2,1e-3,1e-4,1e-5,1e-6])

     inputs_cat = keras.layers.Concatenate(axis=1)(inputs)
     hidden1 = keras.layers.Dense(units=HP_NEURONS, activation=HP_ACTIVATION,kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),kernel_initializer=HP_KERNELINITIALIZER)(inputs_cat)
     hidden2 = keras.layers.Dropout(rate=HP_DROPOUT)(hidden1)
     hidden3 = keras.layers.Dense(units=HP_NEURONS, activation=HP_ACTIVATION,kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),kernel_initializer=HP_KERNELINITIALIZER)(hidden2)
     output = keras.layers.Dense(10,activation="softmax")(hidden3)
     model= keras.Model(inputs=raw_inputs,outputs=output)

#compile model
     metric= tfa.metrics.F1Score(average='macro',num_classes=10)
     loss="categorical_crossentropy"
     optimizer=tf.optimizers.Adam(learning_rate=HP_LEARNINGRATE)
     model.compile(loss =loss,
                              optimizer = optimizer,
                               metrics=metric)
    
     return model

hm = NNHyperModel()



In [9]:
#instantiate Tuner which works with Bayesian Optimization.
tuner = kt.BayesianOptimization(hm,
                     objective=kt.Objective("val_f1_score", direction="max"),
                     max_trials=100
                     )

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
[<KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'string_lookup')>]
[<KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'string_lookup')>, <KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'string_lookup_1')>]
[<KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'string_lookup')>, <KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'string_lookup_1')>, <KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'string_lookup_2')>]
[<KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'string_lookup')>, <KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'string_lookup_1')>, <KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'string_lookup_2')>, <KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'string_lookup_3')>]
[<KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'string_lookup')>, <KerasTensor: shape=(None, 1

INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [10]:
#stop_early has the function to stop training when val_loss  has stopped improving.
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

#start tuner
tuner.search(train_ds,epochs=1000,validation_data=val_ds,callbacks=[stop_early])

INFO:tensorflow:Oracle triggered exit


In [11]:
#get the hyperparamters which performed best on given f1-score.  
best = tuner.get_best_hyperparameters(1)[0]
print(best.values)

{'dropout': 0.0, 'activation': 'elu', 'units': 256, 'kernel_initializer': 'lecun_uniform', 'learning_rate': 0.001}


In [12]:
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name='val_f1_score', direction='max')
Trial summary
Hyperparameters:
dropout: 0.0
activation: elu
units: 256
kernel_initializer: lecun_uniform
learning_rate: 0.001
Score: 0.1640509068965912
Trial summary
Hyperparameters:
dropout: 0.0
activation: elu
units: 256
kernel_initializer: lecun_uniform
learning_rate: 0.001
Score: 0.16240939497947693
Trial summary
Hyperparameters:
dropout: 0.0
activation: elu
units: 256
kernel_initializer: lecun_uniform
learning_rate: 0.001
Score: 0.16231542825698853
Trial summary
Hyperparameters:
dropout: 0.9
activation: selu
units: 256
kernel_initializer: lecun_uniform
learning_rate: 0.0001
Score: 0.16175256669521332
Trial summary
Hyperparameters:
dropout: 0.0
activation: elu
units: 464
kernel_initializer: lecun_uniform
learning_rate: 0.001
Score: 0.16072458028793335
Trial summary
Hyperparameters:
dropout: 0.0
activation: elu
units: 256
kernel_initializer: lecun_uniform
learning_rat